In [0]:
!sudo apt-get install build-essential libasound2-dev libjack-dev

!pip install magenta

# !pip install magenta-gpu

!pip install tensorflow==2.0.0-alpha0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
libjack-dev is already the newest version (1:0.125.0-3).
libasound2-dev is already the newest version (1.1.3-5ubuntu0.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [0]:
from google.colab import files

import magenta.music as mm
import magenta
import tensorflow as tf

In [0]:
path_to_file = tf.keras.utils.get_file('irish.abc', 'https://raw.githubusercontent.com/aamini/introtodeeplearning_labs/2019/lab1/data/irish.abc')

# path_to_file = tf.keras.utils.get_file('irish.abc', 'http://members.yline.com/~zeiler1/abc/czdro.abc')
# path_to_file = tf.keras.utils.get_file('irish.abc', 'http://members.yline.com/~zeiler1/abc/czital.abc')



204800/197618 [===============================] - 0s 0us/step


In [0]:
path_to_file

'/root/.keras/datasets/irish.abc'

In [0]:
with open(path_to_file, 'r') as file:
    raw_data = file.read()

In [0]:
raw_songs = raw_data.split("\n\n")
# for song in raw_songs:
#     print(song)
#     print("Next Song")

In [37]:
print(raw_songs[6])

X:7
T:Boys of Bluehill
Z: id:dc-hornpipe-7
M:C|
L:1/8
K:D Major
dB|BAFA DAFA|BA (3Bcd e2de|fagf egfe|dfed B2dB|!
BAFA DAFA|BA (3Bcd e2de|fagf egfe|d2dc d2:|!
fg|afdf a2gf|gfga b2ag|fagf egfe|dfed B2dB|!
BAFA DAFA|BA (3Bcd e2de|fagf egfe|d2dc d2:|!


In [0]:
songs_wo_tuplets = []

for song in raw_songs:
  if '(' not in song:
    
    # Remove the exclamation mark, which is not supported by the parser
    song_wo_ex = song.replace('!', '')
    songs_wo_tuplets.append(song_wo_ex)
  

In [0]:
test = \
"""
(3ABc|dAFA DFAd|fdcd FAdf|gfge fefd|(3efe (3dcB A2 (3ABc|
dAFA DFAd|fdcd FAdf|gfge fefd|(3efe dc d2:|
AG|FAdA FAdA|GBdB GBdB|Acec Acec|dfaf gecA|
FAdA FAdA|GBdB GBdB|Aceg fefd|(3efe dc d2:|
"""

In [42]:
songs_wo_tuplets[0]

"X:5\nT:Bobby Casey's\nZ: id:dc-hornpipe-5\nM:C|\nL:1/8\nK:A Dorian\ned|cAAB cBcd|eaaf gedB|c2Ac B2Bc|d2de dBGB|\nAGAB cBcd|eaaf gedB|cAAc dBGB|BAAG A2:|\ncd|eaab aged|eaab aged|egga gedB|egga gedB|\nAGAB cBcd|eaaf gedB|cAAc dBGB|BAAG A2:|"

In [56]:
type(res[0][57])

magenta.protobuf.music_pb2.NoteSequence

Refer to documentation on Magenta NoteSequence


https://nikhilsinghmus.github.io/MusicianKit/Structs/NoteSequence.html#/s:11MusicianKit12NoteSequenceV5notesSayAA0C0VGv

In [0]:
notes = res[0][57].notes

In [73]:
type(notes)

google.protobuf.pyext._message.RepeatedCompositeContainer

In [70]:
print(res[0][57].notes)

[pitch: 79
velocity: 90
end_time: 0.25
, pitch: 78
velocity: 90
start_time: 0.25
end_time: 0.5
, pitch: 76
velocity: 90
start_time: 0.5
end_time: 1.0
, pitch: 76
velocity: 90
start_time: 1.0
end_time: 1.25
, pitch: 74
velocity: 90
start_time: 1.25
end_time: 1.5
, pitch: 71
velocity: 90
start_time: 1.5
end_time: 2.0
, pitch: 74
velocity: 90
start_time: 2.0
end_time: 2.25
, pitch: 71
velocity: 90
start_time: 2.25
end_time: 2.5
, pitch: 69
velocity: 90
start_time: 2.5
end_time: 3.0
, pitch: 69
velocity: 90
start_time: 3.0
end_time: 3.25
, pitch: 67
velocity: 90
start_time: 3.25
end_time: 3.5
, pitch: 69
velocity: 90
start_time: 3.5
end_time: 3.75
, pitch: 72
velocity: 90
start_time: 3.75
end_time: 4.0
, pitch: 71
velocity: 90
start_time: 4.0
end_time: 4.25
, pitch: 69
velocity: 90
start_time: 4.25
end_time: 4.5
, pitch: 67
velocity: 90
start_time: 4.5
end_time: 4.75
, pitch: 66
velocity: 90
start_time: 4.75
end_time: 5.0
, pitch: 67
velocity: 90
start_time: 5.0
end_time: 5.25
, pitch: 69


In [52]:
for k in res[0].keys():
  print(k)

57


In [50]:
type(res[0])

dict

In [44]:
# Parse abc to notesequence

from magenta.music import abc_parser as abc_parser

res =  abc_parser.parse_abc_tunebook(songs_wo_tuplets[20])
print(res)

({57: ticks_per_quarter: 220
time_signatures {
  numerator: 2
  denominator: 2
}
key_signatures {
  key: G
}
notes {
  pitch: 79
  velocity: 90
  end_time: 0.25
}
notes {
  pitch: 78
  velocity: 90
  start_time: 0.25
  end_time: 0.5
}
notes {
  pitch: 76
  velocity: 90
  start_time: 0.5
  end_time: 1.0
}
notes {
  pitch: 76
  velocity: 90
  start_time: 1.0
  end_time: 1.25
}
notes {
  pitch: 74
  velocity: 90
  start_time: 1.25
  end_time: 1.5
}
notes {
  pitch: 71
  velocity: 90
  start_time: 1.5
  end_time: 2.0
}
notes {
  pitch: 74
  velocity: 90
  start_time: 2.0
  end_time: 2.25
}
notes {
  pitch: 71
  velocity: 90
  start_time: 2.25
  end_time: 2.5
}
notes {
  pitch: 69
  velocity: 90
  start_time: 2.5
  end_time: 3.0
}
notes {
  pitch: 69
  velocity: 90
  start_time: 3.0
  end_time: 3.25
}
notes {
  pitch: 67
  velocity: 90
  start_time: 3.25
  end_time: 3.5
}
notes {
  pitch: 69
  velocity: 90
  start_time: 3.5
  end_time: 3.75
}
notes {
  pitch: 72
  velocity: 90
  start_time: